In [1]:
import os,sys,glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as st
import functions as fun
sourcedir = '/Volumes/My Passport/cmip5/cmip5'

In [2]:
def append_SF(SF,files,mask2,area):
    if len(files)==0:
        print('nopi SF')
        return SF
        
    for fil in files:
        startyear = np.int(fil[-16:-12])
        if startyear>2100: continue
        print('Added SF',startyear)
        with xr.open_dataset(fil) as ds:
            sf = ds['prsn'].values
        ss = sf[0::12,:,:]
        for f in range(1,12):
            ssnew = sf[f::12,:,:]
            ss = ss[:ssnew.shape[0],:,:]+ssnew
        SF = np.append(SF,np.nansum(ss*mask2*area/12.,axis=(1,2)))
    return SF

In [3]:
def append_TA(TA,files,mask,area):
    if len(files)==0:
        print('nopi SF')
        return TA
    
    for fil in files:
        startyear = np.int(fil[-16:-12])
        if startyear>2100: continue
        print('Added TA',startyear)
        with xr.open_dataset(fil) as ds:
            plev = ds['plev'].values
            idx  = np.argmin((plev-60000)**2)
            ta   = ds['ta'][:,idx,:,:].values
        startmonth=int(fil[-12:-10])
        if startmonth==12:
            junidx = 6
        elif startmonth ==1:
            junidx = 5
        tajun = ta[junidx::12,:,:]
        tajul = ta[junidx+1::12,:,:]
        taaug = ta[junidx+2::12,:,:]
        tajja = (30.*tajun+31.*tajul+31.*taaug)/92.
        weight = mask*area
        if weight.shape[0]>tajja.shape[1]:
            weight = (weight[1:,:]+weight[:-1,:])/2.
        TA = np.append(TA,np.nansum(tajja*weight,axis=(1,2))/np.nansum(weight))
    return TA

In [4]:
def getSF(model,scen,mask2,area):
    
    #Get historical snowfall
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/prsn_Amon_{model}_historical*')
    SF = np.array([])
    SF = append_SF(SF,files,mask2,area)
    lastyear = np.int(files[-1][-9:-5])
    if lastyear>2005: SF = SF[:(2005-lastyear)]
    firstyear = np.int(files[0][-16:-12])

    #Future period
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/prsn_Amon_{model}_{scen}*')
    SF = append_SF(SF,files,mask2,area)
    years = np.arange(firstyear,firstyear+len(SF))

    #picontrol
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/prsn_Amon_{model}_piControl*')  
    SF_pi = np.array([])
    SF_pi = append_SF(SF_pi,files,mask2,area)
    
    #Detrend
    SF = fun.detrend(SF,SF_pi,years)
    
    #Convert to GT/yr
    SF = SF*3.154e-5*1.6
    
    print(scen,model,'Got SF')
    return SF,years

In [5]:
def getTA(model,scen,mask,area):
    #Get historical temperature
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/ta_Amon_{model}_historical*')
    TA = np.array([])
    TA = append_TA(TA,files,mask,area)
    lastyear = np.int(files[-1][-9:-5])
    if lastyear>2005: TA = TA[:(2005-lastyear)]
    firstyear = np.int(files[0][-16:-12])
    
    #Get projected temp
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/ta_Amon_{model}_{scen}*')
    TA = append_TA(TA,files,mask,area)
    years = np.arange(firstyear,firstyear+len(TA))
   
    #No piControl for TA as models show no trends
    
    #Reference
    TA = fun.detrend(TA,np.array([]),years)    
    
    print(scen,model,'Got TA')
    return TA,years

In [6]:
def Gsmb(model,scen):

    #Get Greenland mask
    with xr.open_dataset(f'../rawdata/Gmask/{model}.nc') as ds:
        lon = ds['lon'].values
        lat = ds['lat'].values
        mask = ds['mask'].values
        mask2 = ds['mask2'].values

    #Get surface area
    with xr.open_dataset(f'{sourcedir}/fx/areacella/{model}/r0i0p0/areacella_fx_{model}_historical_r0i0p0.nc') as ds:
        area = ds['areacella'].values

    #Get snowfall change
    SF,years1 = getSF(model,scen,mask2,area)

    #Get temperature increase
    TA,years2 = getTA(model,scen,mask,area)

    #Ensure overlapping periods
    if len(years1)>len(years2):
        SF = SF[np.in1d(years1,years2)]
        years = years2
    elif len(years2)>len(years1):
        TA = TA[np.in1d(years2,years1)]
        years = years1
    else:
        years = years1

    #Convert to sea-level change
    MW  = 84.2*TA + 2.4*TA**2. + 1.6*TA**3.
    SMB = SF-MW
    SLR = -np.cumsum(SMB)*2.8e-4 #in cm
    SLR = fun.detrend(SLR,np.array([]),years)
    
    #Extract 200 year time period 1900-2100
    SLR = SLR[np.logical_and(years>1900,years<2101)]
    years = years[np.logical_and(years>1900,years<2101)]
    
    #Save SLR
    fun.saveSLR(SLR,years,model,scen,'Gsmb')

In [7]:
mods = fun.models()
for scen in ['rcp45','rcp85']:
    for model in mods:
        Gsmb(model,scen)

Added SF 1850
Added SF 2006
Added SF 2100
Added SF 1
rcp45 bcc-csm1-1 Got SF
Added TA 1850
Added TA 2006
Added TA 2100
rcp45 bcc-csm1-1 Got TA
rcp45 bcc-csm1-1 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 2015
rcp45 CanESM2 Got SF
Added TA 1850
Added TA 2006
rcp45 CanESM2 Got TA
rcp45 CanESM2 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 250
Added SF 501
Added SF 800
rcp45 CCSM4 Got SF
Added TA 1850
Added TA 1900
Added TA 1950
Added TA 2006
Added TA 2046
Added TA 2086
rcp45 CCSM4 Got TA
rcp45 CCSM4 Saved  Gsmb
Added SF 1850
Added SF 1900
Added SF 1950
Added SF 2006
Added SF 2056
Added SF 1850
Added SF 1900
Added SF 1950
Added SF 2000
Added SF 2050
Added SF 2100
rcp45 CNRM-CM5 Got SF
Added TA 1850
Added TA 1900
Added TA 1950
Added TA 2006
Added TA 2056
rcp45 CNRM-CM5 Got TA
rcp45 CNRM-CM5 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 1
rcp45 CSIRO-Mk3-6-0 Got SF
Added TA 1850
Added TA 1870
Added TA 1890
Added TA 1910
Added TA 1930
Added TA 1950
Added TA 1970
Added TA 1990
Adde

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


Added SF 11
Added SF 16
Added SF 21
Added SF 26
Added SF 31
Added SF 36
Added SF 41
Added SF 46
Added SF 51
Added SF 56
Added SF 61
Added SF 66
Added SF 71
Added SF 76
Added SF 81
Added SF 86
Added SF 91
Added SF 96
Added SF 101
Added SF 106
Added SF 111
Added SF 116
Added SF 121
Added SF 126
Added SF 131
Added SF 136
Added SF 141
Added SF 146
Added SF 151
Added SF 156
Added SF 161
Added SF 166
Added SF 171
Added SF 176
Added SF 181
Added SF 186
Added SF 191
Added SF 196
Added SF 201
Added SF 206
Added SF 211
Added SF 216
Added SF 221
Added SF 226
Added SF 231
Added SF 236
Added SF 241
Added SF 246
Added SF 251
Added SF 256
Added SF 261
Added SF 266
Added SF 271
Added SF 276
Added SF 281
Added SF 286
Added SF 291
Added SF 296
Added SF 301
Added SF 306
Added SF 311
Added SF 316
Added SF 321
Added SF 326
Added SF 331
Added SF 336
Added SF 341
Added SF 346
Added SF 351
Added SF 356
Added SF 361
Added SF 366
Added SF 371
Added SF 376
Added SF 381
Added SF 386
Added SF 391
Added SF 396
Adde

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


rcp45 MIROC-ESM Got SF
Added TA 1850
Added TA 1990
Added TA 2006
rcp45 MIROC-ESM Got TA
rcp45 MIROC-ESM Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 2000
Added SF 2100
rcp45 MIROC5 Got SF
Added TA 1850
Added TA 1860
Added TA 1870
Added TA 1880
Added TA 1890
Added TA 1900
Added TA 1910
Added TA 1920
Added TA 1930
Added TA 1940
Added TA 1950
Added TA 1960
Added TA 1970
Added TA 1980
Added TA 1990
Added TA 2000
Added TA 2010
Added TA 2006
Added TA 2010
Added TA 2020
Added TA 2030
Added TA 2040
Added TA 2050
Added TA 2060
Added TA 2070
Added TA 2080
Added TA 2090
Added TA 2100
rcp45 MIROC5 Got TA
rcp45 MIROC5 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 1850
Added SF 2036
rcp45 MPI-ESM-LR Got SF
Added TA 1850
Added TA 1860
Added TA 1870
Added TA 1880
Added TA 1890
Added TA 1900
Added TA 1910
Added TA 1920
Added TA 1930
Added TA 1940
Added TA 1950
Added TA 1960
Added TA 1970
Added TA 1980
Added TA 1990
Added TA 2000
Added TA 2006
Added TA 2010
Added TA 2020
Added TA 2030
Added TA 20

...

Added SF 6
Added SF 11
Added SF 16
Added SF 21
Added SF 26
Added SF 31
Added SF 36
Added SF 41
Added SF 46
Added SF 51
Added SF 56
Added SF 61
Added SF 66
Added SF 71
Added SF 76
Added SF 81
Added SF 86
Added SF 91
Added SF 96
Added SF 101
Added SF 106
Added SF 111
Added SF 116
Added SF 121
Added SF 126
Added SF 131
Added SF 136
Added SF 141
Added SF 146
Added SF 151
Added SF 156
Added SF 161
Added SF 166
Added SF 171
Added SF 176
Added SF 181
Added SF 186
Added SF 191
Added SF 196
Added SF 201
Added SF 206
Added SF 211
Added SF 216
Added SF 221
Added SF 226
Added SF 231
Added SF 236
Added SF 241
Added SF 246
Added SF 251
Added SF 256
Added SF 261
Added SF 266
Added SF 271
Added SF 276
Added SF 281
Added SF 286
Added SF 291
Added SF 296
Added SF 301
Added SF 306
Added SF 311
Added SF 316
Added SF 321
Added SF 326
Added SF 331
Added SF 336
Added SF 341
Added SF 346
Added SF 351
Added SF 356
Added SF 361
Added SF 366
Added SF 371
Added SF 376
Added SF 381
Added SF 386
Added SF 391
Added 

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


rcp85 MIROC-ESM Got SF
Added TA 1850
Added TA 1990
Added TA 2006
rcp85 MIROC-ESM Got TA
rcp85 MIROC-ESM Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 2000
Added SF 2100
rcp85 MIROC5 Got SF
Added TA 1850
Added TA 1860
Added TA 1870
Added TA 1880
Added TA 1890
Added TA 1900
Added TA 1910
Added TA 1920
Added TA 1930
Added TA 1940
Added TA 1950
Added TA 1960
Added TA 1970
Added TA 1980
Added TA 1990
Added TA 2000
Added TA 2010
Added TA 2006
Added TA 2010
Added TA 2020
Added TA 2030
Added TA 2040
Added TA 2050
Added TA 2060
Added TA 2070
Added TA 2080
Added TA 2090
Added TA 2100
rcp85 MIROC5 Got TA
rcp85 MIROC5 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 1850
Added SF 2036
rcp85 MPI-ESM-LR Got SF
Added TA 1850
Added TA 1860
Added TA 1870
Added TA 1880
Added TA 1890
Added TA 1900
Added TA 1910
Added TA 1920
Added TA 1930
Added TA 1940
Added TA 1950
Added TA 1960
Added TA 1970
Added TA 1980
Added TA 1990
Added TA 2000
Added TA 2006
Added TA 2010
Added TA 2020
Added TA 2030
Added TA 20

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/xarray/coding/times.py:426: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


rcp85 MRI-CGCM3 Got SF
Added TA 1850
Added TA 1860
Added TA 1870
Added TA 1880
Added TA 1890
Added TA 1900
Added TA 1910
Added TA 1920
Added TA 1930
Added TA 1940
Added TA 1950
Added TA 1960
Added TA 1970
Added TA 1980
Added TA 1990
Added TA 2000
Added TA 2006
Added TA 2016
Added TA 2026
Added TA 2036
Added TA 2046
Added TA 2056
Added TA 2066
Added TA 2076
Added TA 2086
Added TA 2096
rcp85 MRI-CGCM3 Got TA
rcp85 MRI-CGCM3 Saved  Gsmb
Added SF 1850
Added SF 2006
Added SF 700
Added SF 1151
rcp85 NorESM1-M Got SF
Added TA 1850
Added TA 1950
Added TA 2006
rcp85 NorESM1-M Got TA
rcp85 NorESM1-M Saved  Gsmb
